<a href="https://colab.research.google.com/github/rdronamaps/exiftask/blob/master/GCS_Deepforest_Plantation_Automation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install google-cloud-storage

In [2]:
!pip install -U crcmod
!apt-get install python3-crcmod

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 8.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for crcmod: filename=crcmod-1.7-cp310-cp310-linux_x86_64.whl size=31404 sha256=f3e0e69365331bc3c904434b073457a87a6d72335a71c29b44c89b3e80143fc2
  Stored in directory: /root/.cache/pip/wheels/85/4c/07/72215c529bd59d67e3dac29711d7aba1b692f543c808ba9e86
Successfully built crcmod
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  python3-crcmod
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 19.9 kB of archives.
After this operation, 94.2 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 python3-crcmod amd64 1.7+dfsg-2build6 [19.9 kB]
Fetched 19.9 kB in 1s (39.1 kB/s)
Selecting previously unselected package python3-crcmod.
(Reading database ... 123634 files and directories currently i

In [3]:
from google.colab import auth
auth.authenticate_user()

In [4]:
from google.cloud import storage

# Initialize a client
client = storage.Client()

In [5]:
bucket_name = 'dams-prod'
bucket = client.get_bucket(bucket_name)

In [6]:
project_id = 'nhai-poc-sep2023'
!gcloud config set project {project_id}
!gsutil ls

Updated property [core/project].
gs://692634524964-asia-east1-blueprint-config/
gs://ai-ml-output/
gs://dams-dev/
gs://dams-geo-data/
gs://dams-prod/
gs://dams-prod-backup/
gs://dm-recon/
gs://dronamaps-dev/
gs://dronamaps-public/
gs://gcf-v2-sources-692634524964-asia-south1/
gs://gcf-v2-uploads-692634524964.asia-south1.cloudfunctions.appspot.com/
gs://nhaflightplans/


In [7]:
import crcmod
print(crcmod)

<module 'crcmod' from '/usr/local/lib/python3.10/dist-packages/crcmod/__init__.py'>


In [8]:
!pip install deepforest exifread rasterio geopandas shapely opencv-python affine torch rioxarray torchvision numpy matplotlib pandas psutil

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.6/22.6 MB 49.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 53.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.3/819.3 kB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 69.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 543.2/543.2 kB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.5/181.5 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.1/51.1 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 927.3/927.3 kB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 75.2 MB/s eta 0:00:00
   ━━━━━━━━

In [11]:
import os
import warnings
import numpy as np
import geopandas as gpd
import rasterio
from rasterio.io import MemoryFile
from shapely.geometry import Polygon
from deepforest import main
import torch
from google.cloud import storage

# Disable warnings and limit threads
os.environ['JAX_ENABLE_X64'] = 'false'
warnings.filterwarnings("ignore", category=RuntimeWarning)
torch.set_num_threads(1)

# Initialize DeepForest model
model = main.deepforest()
model.use_release()
model.current_device = torch.device("cpu")

# ------------------------------
# GCS utility functions
# ------------------------------
def _split_gcs_uri(gcs_uri):
    """Split gs://bucket/path into (bucket, path)."""
    if not gcs_uri.startswith("gs://"):
        raise ValueError(f"Not a valid GCS URI: {gcs_uri}")
    no_scheme = gcs_uri[5:]  # remove "gs://"
    parts = no_scheme.split("/", 1)
    bucket_name = parts[0]
    if len(parts) == 1:
        prefix = ""
    else:
        prefix = parts[1]
    return bucket_name, prefix

def download_bytes_from_gcs(gcs_uri):
    """Download file bytes from GCS."""
    client = storage.Client()
    bucket_name, path = _split_gcs_uri(gcs_uri)
    bucket = client.bucket(bucket_name)
    blob = bucket.blob(path)
    return blob.download_as_bytes()

def upload_local_file_to_gcs(local_path, gcs_uri):
    """Upload a local file (e.g. .geojson) to GCS."""
    client = storage.Client()
    bucket_name, path = _split_gcs_uri(gcs_uri)
    bucket = client.bucket(bucket_name)
    blob = bucket.blob(path)
    blob.upload_from_filename(local_path)
    print(f"Uploaded {local_path} to {gcs_uri}")

def gcs_path_exists(gcs_uri):
    """
    Check if a GCS object or 'directory' exists by listing or checking the blob.
    For directories, we check if there's at least one object with that prefix.
    """
    client = storage.Client()
    bucket_name, prefix = _split_gcs_uri(gcs_uri)
    bucket = client.bucket(bucket_name)

    # If ends with '/', treat it like a directory prefix
    if prefix.endswith("/"):
        return any(bucket.list_blobs(prefix=prefix, max_results=1))
    else:
        return bucket.blob(prefix).exists()

# ------------------------------
# Main logic
# ------------------------------

def process_file(input_tif_gcs_uri, output_geojson_gcs_uri):
    """
    1) Download TIFF from GCS as bytes
    2) Decode with OpenCV for DeepForest
    3) Open with Rasterio for transform/CRS
    4) Predict
    5) Convert bounding boxes to georeferenced polygons
    6) Save to GeoJSON locally
    7) Upload GeoJSON to GCS
    """
    try:
        # Step 1: Download bytes
        tif_bytes = download_bytes_from_gcs(input_tif_gcs_uri)

        # Step 2: Decode with OpenCV -> image array
        import cv2
        np_arr = np.frombuffer(tif_bytes, np.uint8)
        # - Use IMREAD_UNCHANGED if you want all channels (possibly 4 if RGBA)
        # - Use IMREAD_COLOR if you just want 3-channel BGR
        image = cv2.imdecode(np_arr, cv2.IMREAD_COLOR)
        if image is None:
            raise ValueError("Could not decode TIFF with OpenCV.")

        # Step 3: Extract geoinfo with Rasterio (MemoryFile)
        with MemoryFile(tif_bytes) as mem:
            with mem.open() as src:
                transform = src.transform
                crs = src.crs

        # Step 4: DeepForest predictions
        # If you used IMREAD_COLOR, 'image' will have shape (H, W, 3).
        # If your TIFF has multiple bands, you might only see the first 3 or 4.
        predictions = model.predict_tile(
            image=image,
            return_plot=False,
            patch_size=800,
            patch_overlap=0.2,
            iou_threshold=0.3
        )

        # Step 5: Geolocate bounding boxes
        polygons = []
        scores = []
        for _, row in predictions.iterrows():
            xmin, ymin, xmax, ymax = row['xmin'], row['ymin'], row['xmax'], row['ymax']

            # Rasterio transform: pixel->geographic
            top_left = rasterio.transform.xy(transform, ymin, xmin)
            bottom_right = rasterio.transform.xy(transform, ymax, xmax)

            bbox_polygon = Polygon([
                (top_left[0],     top_left[1]),
                (bottom_right[0], top_left[1]),
                (bottom_right[0], bottom_right[1]),
                (top_left[0],     bottom_right[1]),
                (top_left[0],     top_left[1])
            ])

            polygons.append(bbox_polygon)
            scores.append(row['score'])

        gdf = gpd.GeoDataFrame({'geometry': polygons, 'score': scores}, crs=crs)

        # Step 6: Save to a local temp file, then upload
        temp_geojson = "temp_output.geojson"
        gdf.to_file(temp_geojson, driver='GeoJSON')

        # Step 7: Upload to GCS
        upload_local_file_to_gcs(temp_geojson, output_geojson_gcs_uri)

        # Cleanup
        if os.path.exists(temp_geojson):
            os.remove(temp_geojson)

        print(f"Saved predictions to: {output_geojson_gcs_uri}")

    except Exception as e:
        print(f"Error processing {input_tif_gcs_uri}: {e}")

# -------------------------------------------------------------------
# Example usage dams-prod/ai-ml-input-dataset/N0200220003KA_nov/N0200220003KA_Chunk 1_ortho
# -------------------------------------------------------------------

def process_folder(folder_name, input_dict):
    """
    folder_name: e.g. 'N0400411001MP'
    input_dict: dict of {chunk_number: [list_of_file_numbers]}
    """
    base_path = "gs://dams-prod/Orchestration/November"
    input_base = "gs://dams-prod/ai-ml-input-dataset/N0400401003MP"
    output_base = f"{base_path}/colab_outputs/{folder_name}/plantation"

    if not gcs_path_exists(input_base + "/"):
        print(f"Input folder not found: {input_base}")
        return

    for chunk, file_nums in input_dict.items():
        input_chunk_folder = f"{input_base}/{folder_name}_Chunk {chunk}_ortho"
        output_chunk_folder = f"{output_base}/chunk_{chunk}"

        if not gcs_path_exists(input_chunk_folder + "/"):
            print(f"Folder not found: {input_chunk_folder}")
            continue

        for file_num in file_nums:
            tif_uri = f"{input_chunk_folder}/Chunk_{file_num}.tif"
            geojson_uri = f"{output_chunk_folder}/Chunk_{file_num}.geojson"

            if not gcs_path_exists(tif_uri):
                print(f"TIF not found: {tif_uri}")
                continue

            print(f"Processing: {tif_uri}")
            process_file(tif_uri, geojson_uri)

# Example usage
folders = ["N0400401003MP"]
input_dict = {
    1: list(range(2000)),
    2: list(range(2000)),
    3: list(range(2000))
    # 4: list(range(2000))
}

for folder in folders:
    process_folder(folder, input_dict)


Reading config file: /usr/local/lib/python3.10/dist-packages/deepforest/data/deepforest_config.yml


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


Reading config file: /usr/local/lib/python3.10/dist-packages/deepforest/data/deepforest_config.yml


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


TIF not found: gs://dams-prod/ai-ml-input-dataset/N0400401003MP/N0400401003MP_Chunk 1_ortho/Chunk_0.tif
TIF not found: gs://dams-prod/ai-ml-input-dataset/N0400401003MP/N0400401003MP_Chunk 1_ortho/Chunk_1.tif
TIF not found: gs://dams-prod/ai-ml-input-dataset/N0400401003MP/N0400401003MP_Chunk 1_ortho/Chunk_2.tif
TIF not found: gs://dams-prod/ai-ml-input-dataset/N0400401003MP/N0400401003MP_Chunk 1_ortho/Chunk_3.tif
TIF not found: gs://dams-prod/ai-ml-input-dataset/N0400401003MP/N0400401003MP_Chunk 1_ortho/Chunk_4.tif
TIF not found: gs://dams-prod/ai-ml-input-dataset/N0400401003MP/N0400401003MP_Chunk 1_ortho/Chunk_5.tif
TIF not found: gs://dams-prod/ai-ml-input-dataset/N0400401003MP/N0400401003MP_Chunk 1_ortho/Chunk_6.tif
TIF not found: gs://dams-prod/ai-ml-input-dataset/N0400401003MP/N0400401003MP_Chunk 1_ortho/Chunk_7.tif
TIF not found: gs://dams-prod/ai-ml-input-dataset/N0400401003MP/N0400401003MP_Chunk 1_ortho/Chunk_8.tif
Processing: gs://dams-prod/ai-ml-input-dataset/N0400401003MP/N04

...

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:
Detected KeyboardInterrupt, attempting graceful shutdown ...


Error processing gs://dams-prod/ai-ml-input-dataset/N0400401003MP/N0400401003MP_Chunk 1_ortho/Chunk_117.tif: name 'exit' is not defined
Processing: gs://dams-prod/ai-ml-input-dataset/N0400401003MP/N0400401003MP_Chunk 1_ortho/Chunk_118.tif


KeyboardInterrupt: 